In [12]:
import pickle
import numpy as np
from matplotlib import pyplot as plt      

In [13]:
with open('color_data0_cul_integral.pkl', 'rb') as f:
    colors = pickle.load(f)
with open('color_data1_cul_integral.pkl', 'rb') as f:
    colors_1 = pickle.load(f)

In [14]:
colors

[[array([array([0.00000000e+00, 1.58754863e+16, 3.51276633e+16, 5.13598418e+16,
                2.04866804e+18, 5.35759166e+18, 9.33246217e+18])               ,
         list([2.005363879937967, 12.166111410859045, 25.194943977006197])],
        dtype=object),
  1],
 [array([array([0.00000000e+00, 1.50725347e+16, 3.16179342e+16, 4.74190131e+16,
                7.10943839e+16, 8.36902439e+16, 1.00176999e+17, 1.56594640e+17]),
         list([0.13774838491580965, 39.990552201052964, 24.200362598824405])],
        dtype=object),
  3],
 [array([array([0.00000000e+00, 1.01402336e+16, 1.72389788e+16, 2.20239422e+16,
                4.16823922e+16, 1.61106019e+17, 3.22907666e+17, 5.18004678e+17,
                7.15458044e+17, 7.54327102e+17, 7.84246048e+17, 8.23115106e+17]),
         list([27.40319811581587, 27.546867730618853, 24.59833160933951])],
        dtype=object),
  1],
 [array([array([0.00000000e+00, 1.67183055e+16, 3.99534505e+16]),
         list([28.796820774354728, 26.910831985569

In [15]:
for i in range(len(colors)):
    colors[i][0][0] = colors[i][0][0][-1]

In [16]:
for i in range(len(colors_1)):
    colors_1[i][0][0] = colors_1[i][0][0][-1]

In [17]:
lymans = []
for i in colors:
    lymans.append(i)
for i in colors_1:
    lymans.append(i)

In [18]:
def not_associated(lymans,lower_bound,upper_bound):
    not_associated = []
    for i in lymans:
        if(np.log10(lower_bound<i[0][0] <upper_bound)):
            if(i[1] >1):
                not_associated.append(i)
    return not_associated


In [19]:
def distance(points1, points2):
    return(np.sqrt((points1[0]-points2[0])**2 +(points1[1]-points2[1])**2+(points1[2]-points2[2])**2))

In [20]:
def distance_difference(points1,points2):
    return([(points1[0]-points2[0]),(points1[1]-points2[1]),(points1[2]-points2[2])])

In [21]:
def associated_sphere(all_distance):
    """
    this function finds every galaxy associated with lls in a sphere radius.
    """
    ##going through every los
    distance_from_lls = []
    lyman = []
    galaxies = []
    g = np.loadtxt('hpropsRS.res', usecols=[5,6,7],unpack=1)
    galaxie_location = []
    for i,z,k in zip(g[0],g[1],g[2]):
        galaxie_location.append([i,z,k])
    galaxie_size = np.loadtxt('hpropsRS.res', usecols=[8],unpack=1)
    #lyman
    for i in all_distance:
            for z in range(len(galaxie_location)):
                ## lyman direction from galaxies
                #might need to double check distance convertion(distance away)
                if(distance(i[0][1],galaxie_location[z]) <.15):
                    ##lyman is the all_distance, must includes column density, left,right, middle, peak information, and the last for position
                    ##data is the galaxies information, this includes, halo, postion.
                    lyman.append(i[0][1])
                    galaxies.append([galaxie_size[z],0,0,0,distance(i[0][1],galaxie_location[z])])
                    distance_from_lls.append(distance_difference(galaxie_location[z],i[0][1]))
                    ##this returns, distance difference in projection, halo size, galaxies location, xyz respectively, lyman limited, xyz respectively,
    return lyman,galaxies,distance_from_lls

In [22]:
import scipy as sp
from scipy import spatial
from scipy.stats import binned_statistic

In [23]:
def nfvir3(lyman_distance,n):
    gals = np.loadtxt('hpropsRS.res', usecols=[8],unpack=1) 
    gals_2 =  np.loadtxt('hpropsRS_gcen.res', usecols=[1,2,3],unpack=1) 
    points = []
    for i,j,k in zip(gals_2[0],gals_2[1],gals_2[2]):
        points.append([i,j,k])
    radius_points = []
    for i in gals:
        radius_points.append(i)
    #lls in rvir
    #total lls
    #bins count
    bin_counts = 1
    #one extra space
    points.append([0,0,0])
    lyman_location = []
    galaxies = []
    locations = []
    column_den = []
    for i in lyman_distance:
        #i[0] is the column density, i[1] is the lyman location
        #less than certain density in bins
        points[-1] = (i[1])
        tree = spatial.KDTree(points)
        #look at what is associated
        all_nn_indices = tree.query_ball_point(points[-1],r=2.175e-01*n)
        #check fvir conditions
        #z is the galaxies
        # i see above
        for z in all_nn_indices:
            if(points[z] != points[-1]):
                if(distance(i[1],points[z]) <= radius_points[z]*n):
                    column_den.append(i[0])
                    galaxies.append([radius_points[z],0,0,0,abs(distance(i[1],points[z]))])
                    lyman_location.append(i[1])
                    locations.append([points[z][0]-i[1][0],points[z][1]-i[1][1],points[z][2]-i[1][2]])
    return galaxies,lyman_location,locations, column_den

In [24]:
#check desktop for the right version

def nfvir3_mass(lyman_distance,n):
    gals = np.loadtxt('hpropsRS.res', usecols=[8],unpack=1) 
    gals_2 =  np.loadtxt('hpropsRS_gcen.res', usecols=[1,2,3],unpack=1) 
    gals_mass = np.loadtxt('hpropsRS.res', usecols=[0],unpack=1) 
    points = []
    for i,j,k in zip(gals_2[0],gals_2[1],gals_2[2]):
        points.append([i,j,k])
    radius_points = []
    for i in gals:
        radius_points.append(i)
    #lls in rvir
    #total lls
    #bins count
    bin_counts = 1
    #one extra space
    points.append([0,0,0])
    column_den = []
    mass = []
    location = []
    for i in lyman_distance:
        #i[0] is the column density, i[1] is the lyman location
        #less than certain density in bins
        points[-1] = (i[1])
        tree = spatial.KDTree(points)
        #look at what is associated
        all_nn_indices = tree.query_ball_point(points[-1],r=2.175e-01*n)
        #check fvir conditions
        #z is the galaxies
        # i see above
        for z in all_nn_indices:
            if(points[z] != points[-1]):
                #radius_points[z]*(n-1)<=
                if(radius_points[z]*n-1 <= distance(i[1],points[z]) <= radius_points[z]*n):
                    column_den.append(i[0])
                    location.append(gals_2[z])
                    mass.append(gals_mass[z])
    return column_den,mass,location

unused condition check:            
        condition = False
            for k in all_nn_indices:
                if(points[k] != points[-1]):
                #see if distance in between is smaller than vir distance times by n 
                    if(radius_points[k]*1< distance(i[1],points[k]) < radius_points[k]*2):
                        condition = True
                        #if true
            if condition == True:

In [25]:
def filter(lower,upper,vir,lymans):
    lyman_distance = []
    for i in lymans:
        if(lower<np.log10(i[0][0])<upper):
            if(i[1] == vir):
                lyman_distance.append(i[0])
    return lyman_distance

In [26]:
lyman_distance = filter(19,20,1,lymans)

In [27]:
galaxies, lyman_location,locations, column_den = nfvir3(lyman_distance,5)

KeyboardInterrupt: 

In [ ]:
column_den_1, mass = nfvir3_mass(lyman_distance,2)

In [28]:
def mass_plot(lymans):
    for i in range(1,5):
        lyman_distance = filter(16,20,i,lymans)
        for n in range(i,5):
            column_den,mass = nfvir3_mass(lyman_distance,n)
            data = []
            data.append(column_den)
            data.append(mass)
            name = "mass_" + "lyman_distnace_" +format(i) + "_mass_distance_" + format(n) + ".pkl"
            with open(name, 'wb') as f:
                pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)

In [29]:
mass_plot(lymans)

KeyboardInterrupt: 

In [ ]:
with open('mass_lyman_distnace_1_mass_distance_1.pkl', 'rb') as f:
    data1 = pickle.load(f)
data1

[[], []]

In [ ]:
data = []
data.append(lyman_location)
data.append(galaxies)
data.append(locations)
data.append(column_den)

with open('17-18_within_2_gcen_cul_int_new.pkl', 'wb') as f:
    pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
gals = np.loadtxt('hpropsRS.res', usecols=[8],unpack=1) 
gals_2 =  np.loadtxt('hpropsRS_gcen.res', usecols=[1,2,3],unpack=1) 
points = []
for i,j,k in zip(gals_2[0],gals_2[1],gals_2[2]):
    points.append([i,j,k])
radius_points = []
for i in gals:
    radius_points.append(i)
    

In [ ]:
gals = np.loadtxt('hpropsRS.res', usecols=[0],unpack=1) 
mass = []
for i in gals:
    mass.append(i)
for i in range(len(radius_points)):
    if(distance([12.285990942182934, 4.29564598784834, 35.748367252964826],points[i]) <= radius_points[i]*1):
        print(i)
        print(mass[i])
        print(radius_points[i])
        print(points[i])

31872
2756000000.0
0.03172
[12.28392, 4.30908, 35.74278]
